# EnergyNation — MPI Risk Engine **v02** (Colab One-Click)

This notebook runs the **v02** MPI risk engine in Google Colab. It:
- Installs dependencies,
- Writes `mpi_risk_engine_v02.py` into the runtime,
- Prompts you to upload the three input files,
- Executes the engine and saves outputs (`mpi_2024_scored.csv/.xlsx`).

**Required inputs (upload when prompted):**
- `bayes_lr_regenerated_coefficients.csv`
- `cox_coefficients_with_references.csv`
- `mpi_2024_input.xlsx`


## 1) Install dependencies

In [ ]:
#@title Install baseline packages
!pip -q install --upgrade pip
!pip -q install pandas numpy openpyxl lifelines scikit-learn

## 2) Write the engine script (`mpi_risk_engine_v02.py`) into runtime

In [ ]:
#@title Create engine script in runtime
engine_path = "/content/mpi_risk_engine_v02.py"
engine_code = """
# -*- coding: utf-8 -*-
\"\"\"mpi_risk_engine_v02.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1rHV3imTYWy9_bjWBdWGl_XeQ6VklloA-
\"\"\"

# -*- coding: utf-8 -*-
\"\"\"Copy of Welcome To Colab

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1qiYWYtjN97waW4i4kF93neqTtg6J-TIe

# New Section
\"\"\"

import pandas as pd
import numpy as np

BAYES_COEFF_PATH = "/content/bayes_lr_regenerated_coefficients.csv"
COX_COEFF_PATH   = "/content/cox_coefficients_with_references.csv"
INPUT_XLSX       = "/content/mpi_2024_input.xlsx" # Corrected path and filename
OUT_CSV          = "/content/mpi_2024_scored.csv"
OUT_XLSX         = "/content/mpi_2024_scored.xlsx"

S0_3Y = 0.9491  # baseline survival at 3 years (confirmed)

def norm_str(x):
    if pd.isna(x):
        return "Unknown"
    s = str(x).strip()
    return s if s else "Unknown"

def start_bin_from_year(y):
    try:
        y = int(y)
    except:
        return "Unknown"
    return str(y) if 2018 <= y <= 2024 else "Unknown"

def run():
    df = pd.read_excel(INPUT_XLSX)

    bayes_table = pd.read_csv(BAYES_COEFF_PATH)
    cox_table = pd.read_csv(COX_COEFF_PATH)

    bayes_lr_map = dict(zip(bayes_table["feature_name"], bayes_table["LR"]))
    cox_coef_map = dict(zip(cox_table["covariate"], cox_table["coef"]))

    # === Bayes: derive cost quintile from project_cost via qcut (dataset-relative) ===
    ranks = df["project_cost"].astype(float).rank(method="first")
    df["_cost_quintile_bayes"] = pd.qcut(ranks, 5, labels=[0,1,2,3,4]).astype("Int64")

    # === Cox: derive cost quintile from cost_percentile (0-1) -> int(p*5) ===
    def cost_quintile_for_cox(p):
        if pd.isna(p): return None
        v = float(p)
        return int(min(0.9999, max(0.0, v))*5)

    # --- p_bayes ---
    def compute_p_bayes(row):
        feats = []
        # cleantech
        cle = norm_str(row.get("cleantech"))
        feats.append(f"cleantech_{cle if cle in ['Yes','No'] else 'Unknown'}")
        # cost quintile (from qcut)
        cq = row.get("_cost_quintile_bayes")
        feats.append(f"cost_quintile_{int(cq) if pd.notna(cq) else 'Unknown'}")
        # group
        grp = norm_str(row.get("group"))
        feats.append(f"group_{grp}" if f"group_{grp}" in bayes_lr_map else "group_Unknown")
        # province
        prov = norm_str(row.get("province"))
        feats.append(f"province_{prov}" if f"province_{prov}" in bayes_lr_map else "province_Unknown")
        # sector
        sec = norm_str(row.get("sector"))
        feats.append(f"sector_{sec}" if f"sector_{sec}" in bayes_lr_map else "sector_Unknown")
        # start year bin
        sb = start_bin_from_year(row.get("start_year"))
        feats.append(f"start_bin_{sb}" if f"start_bin_{sb}" in bayes_lr_map else "start_bin_Unknown")
        # province-sector interaction
        ps = f"prov_sec_{prov}_{sec}"
        if ps in bayes_lr_map:
            feats.append(ps)

        log_odds = sum(np.log(bayes_lr_map.get(f, 1.0)) for f in feats if f in bayes_lr_map)
        odds = np.exp(log_odds)
        return odds / (1.0 + odds)

    # --- Cox ---
    def compute_risk_score(row):
        eta = 0.0
        # cleantech_flag
        cle = norm_str(row.get("cleantech"))
        cle_v = 1 if cle == "Yes" else 0
        eta += cox_coef_map.get("cleantech_flag", 0.0) * cle_v
        # cost_quintile numeric from cost_percentile
        cq = cost_quintile_for_cox(row.get("cost_percentile"))
        if cq is not None:
            eta += cox_coef_map.get("cost_quintile", 0.0) * int(cq)
        # province & sector
        prov = norm_str(row.get("province"))
        sec = norm_str(row.get("sector"))
        eta += cox_coef_map.get(f"province_{prov}", 0.0)
        eta += cox_coef_map.get(f"sector_{sec}", 0.0)
        return float(np.exp(eta))

    df["p_bayes"] = df.apply(compute_p_bayes, axis=1)
    df["risk_score"] = df.apply(compute_risk_score, axis=1)
    df["years_remaining"] = (5.0 - df["reporting_years"]).clip(lower=0.25)
    df["p_cox"] = 1 - (S0_3Y ** df["risk_score"])
    df["blended_prob"] = 0.60 * df["p_bayes"] + 0.40 * df["p_cox"]
    df["priority_index"] = df["blended_prob"] / df["years_remaining"]

    # Rescale within filtered dataset
    pi_min = df["priority_index"].min()
    pi_max = df["priority_index"].max()
    df["urgency_scale_(0-1)"] = (df["priority_index"] - pi_min) / (pi_max - pi_min) if pi_max > pi_min else 0.0

    df["power_ranking"] = 0.60 * df["blended_prob"] + 0.40 * df["urgency_scale_(0-1)"]

    # Save
    df.to_csv(OUT_CSV, index=False)
    df.to_excel(OUT_XLSX, index=False)
    return df

if __name__ == "__main__":
    out = run()
    print("Wrote:", OUT_CSV)
    print("Wrote:", OUT_XLSX)
    print("Rows:", len(out))

# List the files in the /content directory
import os
os.system("ls /content/")
"""
with open(engine_path, "w", encoding="utf-8") as f:
    f.write(engine_code)
print("Wrote:", engine_path)
!sed -n '1,80p' {"engine_path"}
print("\n... (script truncated) ...")

## 3) Upload required input files (or mount Drive in 3b)

In [ ]:
#@title Upload from your computer
from google.colab import files
print("Select the three input files:")
uploaded = files.upload()
print("Uploaded:", list(uploaded.keys()))
!ls -lh /content/ | sed -n '1,80p'

## 3b) (Optional) Mount Google Drive

In [ ]:
#@title Mount Drive (optional)
from google.colab import drive
drive.mount('/content/drive')
!ls -lh /content/drive/MyDrive | sed -n '1,80p'

## 4) Run the engine
Expected filenames in `/content/`:
- `bayes_lr_regenerated_coefficients.csv`
- `cox_coefficients_with_references.csv`
- `mpi_2024_input.xlsx`

In [ ]:
#@title Execute engine
!python /content/mpi_risk_engine_v02.py || true
!echo "\nFiles in /content after run:" && ls -lh /content | sed -n '1,120p'

## 5) Preview outputs (head)

In [ ]:
#@title Preview outputs
import pandas as pd, os
for fn in ["/content/mpi_2024_scored.csv", "/content/mpi_2024_scored.xlsx"]:
    if os.path.exists(fn):
        try:
            df = pd.read_csv(fn) if fn.endswith('.csv') else pd.read_excel(fn)
            display(df.head(10))
        except Exception as e:
            print("Could not preview", fn, e)

## 6) Download results

In [ ]:
#@title Download outputs
from google.colab import files
for fn in ["/content/mpi_2024_scored.csv", "/content/mpi_2024_scored.xlsx"]:
    try:
        files.download(fn)
    except Exception as e:
        print("Could not download:", fn, e)